In [1]:
import os
from dotenv import load_dotenv
from langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper
from langchain_community.agent_toolkits.financial_datasets.toolkit import FinancialDatasetsToolkit
from langchain_openai import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

/Users/fernandacladeramelgar/Documents/2_Projects/00_KGFP/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Load environment variables
load_dotenv()
os.environ["FINANCIAL_DATASETS_API_KEY"] = os.getenv("FINANCIAL_DATASETS_API_KEY")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# Initialize API and Toolkit
api_wrapper = FinancialDatasetsAPIWrapper(financial_datasets_api_key=os.getenv("FINANCIAL_DATASETS_API_KEY"))
toolkit = FinancialDatasetsToolkit(api_wrapper=api_wrapper)

In [4]:
# Neo4j Graph Setup
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=False
)

/var/folders/_z/vq1q1pzj639b28jbhlxzd0zw0000gn/T/ipykernel_16130/3669920713.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
# System Prompt for LangChain Model
system_prompt = """
You are an advanced financial analysis AI assistant integrated with APIs and Neo4j Knowledge Graphs.

Capabilities:
1. Retrieve financial data (e.g., balance sheets, income statements, cash flows).
2. Use Neo4j to analyze relationships and correlations.
3. Combine multi-source data to provide actionable insights.

Always explain your reasoning, specify data sources, and provide concise, actionable insights.
"""

In [6]:
# Initialize LLM Model and Prompt
model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [7]:
# Query Executor Function
def query_executor(input_data):
    source = input_data.get("source", "")
    ticker = input_data.get("ticker", "")

    if "financial_api" in source:
        return api_wrapper.get_income_statements(ticker=ticker)
    elif "neo4j" in source:
        return graph.query(f"MATCH (n:Stock {{ticker: '{ticker}'}}) RETURN n")
    else:
        return {"error": "Unsupported source"}

In [8]:
# Financial Metrics Schema
class FinancialMetrics(BaseModel):
    revenue: str = Field(..., description="The company's revenue.")
    debt: str = Field(..., description="The company's total debt.")

# Workflow for Parallel Data Retrieval
financial_workflow = RunnableParallel(
    {
        "api_data": RunnableLambda(lambda x: api_wrapper.get_balance_sheets(ticker=x["ticker"])),
        "graph_data": RunnableLambda(lambda x: graph.query(f"MATCH (n:Stock {{ticker: '{x['ticker']}'}}) RETURN n")),
    }
)

In [9]:
# LangChain Agent Setup
agent = create_tool_calling_agent(model, toolkit.get_tools(), prompt)
agent_executor = AgentExecutor(agent=agent, tools=toolkit.get_tools())

# Example Query
query = "What was AAPL's revenue in 2023 and its debt in Q1 2024?"
input_data = {
    "input": query,
    "agent_scratchpad": ""
}
response = agent_executor.invoke(input_data)
print(response)

{'input': "What was AAPL's revenue in 2023 and its debt in Q1 2024?", 'agent_scratchpad': '', 'output': "In 2023, Apple's (AAPL) revenue was $391.035 billion USD. In Q1 2024, Apple's total debt was $106.629 billion USD. \n\nThese figures provide insight into Apple's financial performance and leverage during this period, indicating both its revenue-generating capability and its level of indebtedness."}


In [10]:
# Embedding-Based Similarity Search with Neo4j
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vector_index = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    search_type="hybrid",
    node_label="Stock",
    text_node_properties=["description"],
    embedding_node_property="embedding"
)
search_results = vector_index.similarity_search("Find tech companies with high revenue.")
print(search_results)

/var/folders/_z/vq1q1pzj639b28jbhlxzd0zw0000gn/T/ipykernel_16130/2126654436.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({nod

[]
